## Code 3-1 to Code 3-9

In [1]:
import pandas as pd
import stop_words
import re
from nltk.tokenize import sent_tokenize

In [2]:
t1 = pd.read_csv("Reviews.csv")

In [3]:
t1.shape

(568454, 10)

In [4]:
t1.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


;Citing Lexicons
;
1. Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." 
;       Proceedings of the ACM SIGKDD International Conference on Knowledge  ;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, 
;       Washington, USA, 
2. Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing 
;       and Comparing Opinions on the Web." Proceedings of the 14th 
;       International World Wide Web conference (WWW-2005), May 10-14, 
;       2005, Chiba, Japan.

In [5]:
pos1 = pd.read_csv("positive-words.txt",sep="\t",error_bad_lines=True,encoding='latin1',header=None)
neg1 = pd.read_csv("negative-words.txt",sep="\t",error_bad_lines=True,encoding='latin1',header=None)

pos1.columns = ["words"]
neg1.columns = ["words"]

pos_set = set(list(pos1["words"]))
neg_set = set(list(neg1["words"]))

In [6]:
# In our “t1” dataset we have 2 columns of text - “Text” and “Summary”. 
# We would now combine them and process them into a single column. 
# It is this column that would go through the lexicon mining

In [ ]:
t1["full_txt"] = t1["Summary"] + " " + t1["Text"]
t1["full_txt"] = t1["full_txt"].str.lower()
t1["sent_len"] = t1["full_txt"].str.count(" ") + 1


In [ ]:
t2 = t1[t1.sent_len>=1]
len(t1),len(t2)

In [14]:
##for meausring accuracy
t2["score_bkt"]="neu"
t2.loc[t2.Score>=4,"score_bkt"] = "pos"
t2.loc[t2.Score<=2,"score_bkt"] = "neg"

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [15]:
t3 = t2.sample(frac=0.1)
len(t2),len(t3)

(568427, 56843)

To iterate through all words in a sentence corpus and hit against the list of lexicons. 
Since these are longer sentences we would want to normalize the number of positive and negative hits by number of words in the sentences. 
After this a simple comparison between the positive, negative and neutral scores is done and the sentences are tagged based on whichever scores are higher

In [16]:
final_tag_list = []
pos_percent_list = []
neg_percent_list = []
pos_set_list = []
neg_set_list = []

for i,row in t3.iterrows():
    
    full_txt_set = set(row["full_txt"].split())
    sent_len = len(full_txt_set)
    
    pos_set1 = (full_txt_set) & (pos_set)
    neg_set1 = (full_txt_set) & (neg_set)
    
    com_pos = len(pos_set1)
    com_neg = len(neg_set1)
    
    if(com_pos>0):
        pos_percent = com_pos/sent_len
    else:
        pos_percent = 0

    
    if(com_neg>0):
        neg_percent = com_neg/sent_len
    else:
        neg_percent =0
        
    if(pos_percent>0)|(neg_percent>0):
        if(pos_percent>neg_percent):
            final_tag = "pos"
        else:
            final_tag = "neg"
    else:
        final_tag="neu"
    
    final_tag_list.append(final_tag)
    pos_percent_list.append(pos_percent)
    neg_percent_list.append(neg_percent)
    pos_set_list.append(pos_set1)
    neg_set_list.append(neg_set1)


In [17]:
t3["final_tags"] = final_tag_list
t3["pos_percent"] = pos_percent_list
t3["neg_percent"] = neg_percent_list

t3["pos_set"] = pos_set_list
t3["neg_set"] = neg_set_list

In [18]:
from sklearn.metrics import accuracy_score
print (accuracy_score(t3["score_bkt"],t3["final_tags"]))

0.7665675632883556


In [19]:
from sklearn.metrics import f1_score
f1_score(t3["score_bkt"],t3["final_tags"], average='macro')  

0.4628366707118743

In [14]:
rows_name = t3["score_bkt"].unique()

from sklearn.metrics import confusion_matrix
cmat = pd.DataFrame(confusion_matrix(t3["score_bkt"],t3["final_tags"], labels=rows_name, sample_weight=None))
cmat.columns = rows_name 
cmat["act"] = rows_name
cmat

,pos,neu,neg,act
0,38968,500,4880,pos
1,3024,125,1181,neu
2,3612,305,4248,neg


In [2]:
##we first investigate the errors by comparing the predicted sentiment and the actual tags

In [15]:
pd.options.display.max_colwidth=1000
t3.loc[t3.score_bkt!=t3.final_tags,["Summary","full_txt","final_tags","score_bkt","pos_percent","neg_percent","pos_set","neg_set"]]

,Summary,full_txt,final_tags,score_bkt,pos_percent,neg_percent,pos_set,neg_set
558552,A 20 year romance.,"a 20 year romance. a friend gave me my first flake over 20 years ago. her father had brought home a precious handful of them from a business trip to the uk. boy, she must have really loved me to give me one of my very own!<br /><br />cadbury makes good chocolate, but what sets these apart is how they are made. of course - they are flaky. they are so fragile, they will fall apart in your hand really easily. because of this fragility, they also melt in your mouth very nicely.<br /><br />one of the best (and i think this may be traditional) ways to indulge with a flake is to stab it into a malt or milk shake and then let it fall apart. yum!",neg,pos,0.042553,0.053191,"{precious, loved, good, best}","{indulge, fall, flake, shake, stab}"
524726,very good,very good i have been eating sardines for about a year. brunswick in oil is my favorite. i tryed king oscar and they were good. i would rate these as almost as good...but i payed $1 per can and they're good for 4 1/2 years!!! in my opinion brunswick makes the best sardines for the money. i would buy from a local store when they are on sale!!! i just bought 35 cans tonight....the cashier always gives me a funny look...but it doesn't bother me!!! (3/13/12 big fat paulyyy),neg,pos,0.028571,0.042857,"{good, best}","{bother, fat, funny}"
33599,great pocket candy,"great pocket candy these have pretty good flavor which doesn't disappear after the first minute. they are stickier than charms sour balls or lifesavers, but they are each individually wrapped so they don't stick to each other and half a roll doesn't grab all your pocket lint.",pos,neu,0.073171,0.024390,"{good, pretty, great}",{sour}
241561,hope it will live,hope it will live the seller was fast to ship and at this time the plant looks ok .... but it is way to large for such small pot roots were all taped to the side of the pot ... and i broke many trying to remove all the tape and now i am not sure it will live ... we will see,neg,neu,0.022222,0.022222,{fast},{broke}
317727,Absolutely wonderful!,absolutely wonderful! i ordered this product after having another brand of apple cider and needless to say i'm impressed. i will definitely order another box shortly as it's going quick! the flavor is excellent and you can taste the caramel!,neg,pos,0.027778,0.027778,{excellent},{needless}
197839,Need some cream and sugar,"need some cream and sugar i drink my coffee black and like some flavor now and then, but with this i need to add cream and sugar to enjoy it. but, when i do add creamer, it is very nice.",pos,neu,0.068966,0.000000,"{like, enjoy}",{}
311964,Wouldn't order again,"wouldn't order again i eat a lot at my desk, so was looking for healthy snacking options. i love kale chips so i thought i'd give the other chips a try. these certainly taste healthy, by that i mean, they're not the sort of ""these taste guilty"" snack that kale chips are. these taste like carrots and beets; the cheddar flavor doesn't really come through. these chips also aren't entirely crunchy, they're maybe a little chewy. i will eat them, they're not so bad that i will just throw them out. but, i wouldn't order these again.",pos,neu,0.044776,0.014925,"{love, like, healthy}",{bad}
20451,"Speaking for ""The Cat"" - This Stuff is Great!","speaking for ""the cat"" - this stuff is great! what can i say? the cat loved it and lapped every bit of it up from her bowl. she is usually a picky eater, but this suited her fancy. what more can you ask of a cat food!",neg,pos,0.026316,0.026316,{loved},{picky}
112490,They're not bad,"they're not bad let me begin by saying that i do not like oreo cookies at all. i think the cookie part is bitter and i hate the white cream filling. when a friend suggested i try these i was reluctant and kept telling her that i was not interested. she insisted so i tried a few and surprisingly did not find them to be bad at all. i'm diabet

In [3]:
### Accuracy without neutral to understand performance better

In [20]:
t4 = t3.loc[(t3.score_bkt!="neu") & (t3.final_tags!="neu")].reset_index()
print (accuracy_score(t4["score_bkt"],t4["final_tags"]))
print (f1_score(t4["score_bkt"],t4["final_tags"],average='macro'))

0.8391070945815471
0.7085794863623065
